In [ ]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib
#from mpl_toolkits.mplot3d import Axes3D
from gen_mat import *
from block_rect_maxvol import *
import re
import os
%matplotlib inline

In [ ]:
dir_str = '.'
nder = 2 # 
expansion = 4
n = 30 # number of points
n_test = 50000 #points on test grid
p_size = (nder+1)*expansion #number of monoms (number of columns)
x = 2*np.random.rand(n, nder)-1
np.savez(os.path.join(dir_str,'taken_points'), x=x)

In [ ]:
c = GenMat(p_size, x,poly=cheb, debug=False,pow_p=1)
# c = GenMat(p_size, x, poly=trigpoly, poly_diff=trigpoly_diff, debug=False, pow_p=1)
print c.shape
# turn matrix to the block structure
A = matrix_prep(c,nder+1,n)
print A.shape

### Testing the new most general environment

In [ ]:
### generating test points
points_test = test_points_gen(n_test, nder, distrib='random') 
### evaluating test
max_row = 10
functions = [rosenbrock,sincos,roots,gauss]
for function in functions:
    with open("func={}.txt".format(function.__name__),"w+") as f:
        with  open("points_func={}.txt".format(function.__name__),"w+") as f1:
            for expansion in range(1,4):
                for N_rows_ex in range(expansion, max_row):
                    N_rows = N_rows_ex*(nder+1)
                    try:
                        error, taken_points = test(A, x, points_test, nder, expansion, N_rows, function)
                    except SingularError:
                        print ('not full column rank with expansion={}'.format(expansion)) 
                        continue
                    f.write("error={}_Nrows={}_expans={}\n".format(error, N_rows, expansion))
                    taken_points.tofile(f1,sep=" ")
                    f1.write("_error={}_expansion={}\n".format(error,expansion))
                    f.flush()
                    f1.flush()

In [ ]:
def file_extraction_old(Filepath):
    f = open(Filepath)
    line = f.readline()
    error, N_rows, N_col =[],[],[]
     
    while line != "":
        pos1 = line.find('error=') + len('error=')
        pos2 = line.find('_Nrows=')
        pos3 = line.find('_expans=')
        pos4 = line.find('\n')
        error.append(float(line[pos1:pos2]))
        N_rows.append(int(line[pos2+len('_Nrows='):pos3]))
        N_col.append(int(line[pos3+len('_expans=') :pos4]))
        line = f.readline()
    f.close()
    return (error, N_rows, N_col)

def file_extraction(Filepath):
    srch = re.compile(r'error=([\+\-\d\.eE]+)_Nrows=(\d+)_expans=(\d+)')
    fnd = srch.findall(open(Filepath, 'r').read())
    return tuple(np.array(i) for i in zip(*[(float(i0), int(i1), int(i2)) for i0, i1, i2 in fnd]))


def DataToMesh(error, N_row, N_col):
    row_s = sorted(list(set(N_row)))
    col_s = sorted(list(set(N_col)))
    data = {}
    for i, e in enumerate(error):
        data[(N_row[i], N_col[i])] = e
    
    res = np.empty((len(row_s), len(col_s)), dtype=float)
    for i, r in enumerate(row_s):
        for j, c in enumerate(col_s):
            try:
                res[i,j] = data[(r, c)]
            except:
                res[i,j] = np.nan
    X, Y = np.meshgrid(row_s, col_s)
    return res.T, X, Y

In [ ]:
error, N_row, N_col = DataToMesh(*file_extraction("func=Gauss.txt"))
fig = plt.figure()
ax = fig.gca(projection='3d')
# N_row, N_col = np.meshgrid(N_row, N_col)
ax.plot_surface(N_row, N_col, error, edgecolor='black', linewidth=2)
ax.legend()
ax.set_xlabel('N_rows', fontsize=10)
ax.set_ylabel('N_columns')
plt.show()